# Setup

Before you can begin using the Translation API, run the following command in Cloud Shell to enable the API:

In [1]:
!gcloud services enable translate.googleapis.com

Get the project id (to be used in your application):

In [2]:
!echo $(gcloud config get-value core/project)

truster-tester-program


Install IPython and the Translation API client library:

In [3]:
!pip install google-cloud-translate

In [4]:
from os import environ

from google.cloud import translate

In [5]:
PROJECT_ID = 'truster-tester-program'
PARENT = f"projects/{PROJECT_ID}"

The PARENT variable will be used for requests in the next steps. It refers to the caller's project.

# List available languages

In [6]:
def print_supported_languages(display_language_code: str):
    client = translate.TranslationServiceClient()

    response = client.get_supported_languages(
        parent=PARENT,
        display_language_code=display_language_code,
    )

    languages = response.languages
    print(f" Languages: {len(languages)} ".center(60, "-"))
    for language in languages:
        language_code = language.language_code
        display_name = language.display_name
        print(f"{language_code:10}{display_name}")

In [7]:
print_supported_languages("en")

---------------------- Languages: 194 ----------------------
ab        Abkhaz
ace       Acehnese
ach       Acholi
af        Afrikaans
sq        Albanian
alz       Alur
am        Amharic
ar        Arabic
hy        Armenian
as        Assamese
awa       Awadhi
ay        Aymara
az        Azerbaijani
ban       Balinese
bm        Bambara
ba        Bashkir
eu        Basque
btx       Batak Karo
bts       Batak Simalungun
bbc       Batak Toba
be        Belarusian
bem       Bemba
bn        Bengali
bew       Betawi
bho       Bhojpuri
bik       Bikol
bs        Bosnian
br        Breton
bg        Bulgarian
bua       Buryat
yue       Cantonese
ca        Catalan
ceb       Cebuano
ny        Chichewa
zh        Chinese (Simplified)
zh-CN     Chinese (Simplified)
zh-TW     Chinese (Traditional)
cv        Chuvash
co        Corsican
crh       Crimean Tatar
hr        Croatian
cs        Czech
da        Danish
dv        Dhivehi
din       Dinka
doi       Dogri
dov       Dombe
nl        Dutch
dz        Dzongkha


In [ ]:
print_supported_languages("de")

# Translate text

You can use the Translation API to translate text from one language to another. Text is translated using the Neural Machine Translation (NMT) model. If the NMT model is not supported for the requested language translation pair, the Phrase-Based Machine Translation (PBMT) model is used. For more on Google Translate and its translation models, see the NMT announcement post.

In [8]:
def translate_text(text: str, target_language_code: str) -> translate.Translation:
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=PARENT,
        contents=[text],
        target_language_code=target_language_code,
    )

    return response.translations[0]
    

Call the function to translate the same text in different languages:



In [9]:
text = "Hello World!"
target_languages = ["tr", "de", "es", "it", "el", "zh", "ja", "ko"]

print(f" {text} ".center(50, "-"))
for target_language in target_languages:
    translation = translate_text(text, target_language)
    source_language = translation.detected_language_code
    translated_text = translation.translated_text
    print(f"{source_language} → {target_language} : {translated_text}")
    

------------------ Hello World! ------------------
en → tr : Selam Dünya!
en → de : Hallo Welt!
en → es : ¡Hola Mundo!
en → it : Ciao mondo!
en → el : Γεια σου Κόσμο!
en → zh : 你好世界！
en → ja : 「こんにちは世界」
en → ko : 안녕하세요!


# Translate an article

In [10]:
text = """
Die Staatsanwaltschaft ermittelt nach dem Tod der 33-jährigen Surferin an der Eisbachwelle im Englischen Garten in München. Im Moment darf niemand mehr ins Wasser. Aber ist Surfen dort grundsätzlich gefährlich? Gefährlicher als Surfen im Meer?

Zum Artikel: "Schwer Glück gehabt": Surfer erzählt von eigenem Unfall an Isar
Erfahrene Eisbach-Surfer sagen, es sei ein großer Unterschied, ob man in einem Fluss surft oder im Meer. Die Welle komme aus der anderen Richtung und habe einen Auftrieb, sagt Sophia. Sie surft seit neun Jahren an der Eisbachwelle und steht dort zwei bis drei Mal in der Woche auf dem Surfbrett.

Eisbachwelle kein Anfänger-Spot

Die Eisbachwelle sei aber beileibe keine Surf-Spot für Anfänger, erklärt der BR-Journalist Mike Lingenfelser, der dort seit den Anfängen der Welle in den 1980er-Jahren surft. Nur erfahrene Surfer sollten dort ins Wasser gehen, sagt er. Ganz wichtig ist für ihn dabei der Umgang mit der Leash, also der Sicherungsleine. Die Leash ist eine Leine, die den Surfer mit seinem Brett verbindet. Sie wird mit einem Klettverschluss am Körper befestigt, meistens am Unterschenkel, damit das Brett nicht abtreibt, wenn man absteigt oder herunterfällt.

Laut Lingenfelser muss man in der Lage sein, seine Leash blind unter Wasser öffnen zu können. Man könne das auch mal als Trockenübung im Wohnzimmer versuchen, erklärt der Hobbysurfer. Die Augen schließen, mit der Hand bis zum Knöchel vortasten und dann versuchen, dieses Sicherungsseil, an dem es eine Reißlasche gibt, mit einem Griff zu öffnen.

Schutz per Helm und Weste

Aber man kann noch mehr machen, um sich zu schützen – zum Beispiel einen Helm tragen, wie die Eisbachsurferin Sophia. Ihr fiel das Brett schon einmal auf den Kopf. Man kann auch noch eine Weste tragen, die Auftrieb gibt, wenn man ins Wasser fällt. Ganz wichtig aus Lingenfelsers Sicht: Man solle nie allein surfen. Und: Immer aufmerksam schauen, wo die anderen Surfer gerade sind und ob sie nach einem Sturz vom Brett wieder auf der Wasseroberfläche auftauchen.

Kein hundertprozentiger Schutz – Training senkt Risiko

Aber eine hundertprozentige Sicherheit gebe es bei diesem Sport nicht, sagt Lingenfelser. Es bleibe bei Extremsportarten ein Restrisiko, das man vielleicht nicht zu hundert Prozent ausschalten kann. Man könne es minimieren durch viel Erfahrung, durch Training und bestimmte Sicherheitsmaßnahmen.

Die Eisbachwelle gibt es seit rund 40 Jahren, und natürlich hätten sich Surfer dort immer wieder mal verletzt, wie beide Surfer berichten. Aber schwere oder tödliche Unfälle hatte es bis jetzt in München nicht gegeben. Die Surfer hoffen deshalb, dass, wenn die Untersuchungen der Staatsanwaltschaft beendet sind, die Welle von der Stadt München wieder freigegeben wird.

Eisbach wird jetzt abgesenkt

Am Mittwoch – zwei Wochen nach dem Unglück – soll das Wasser am Eisbach abgesenkt werden, wie die Staatsanwaltschaft München I mitteilte. Von dieser Maßnahme erhofft man sich, mehr über die Todesursache der jungen Surferin zu erfahren.

Ihre Leash hatte sich aus unbekannten Gründen am Grund des Eisbachs verhakt. Die Frau konnte sich wegen der starken Strömung nicht selbst befreien. Auch andere Surfer scheiterten. Erst Feuerwehrleuten gelang es, die Leine mit einem Tauchermesser zu durchtrennen und die Frau so zu befreien. Woran sich die Leine oder das Brett verfangen haben könnte, ist bislang unbekannt."""

In [11]:
target_languages = ["pl"]

In [12]:
print(f" {text} ".center(50, "-"))
for target_language in target_languages:
    translation = translate_text(text, target_language)
    source_language = translation.detected_language_code
    translated_text = translation.translated_text
    print(f"{source_language} → {target_language} : {translated_text}")
    

 
Die Staatsanwaltschaft ermittelt nach dem Tod der 33-jährigen Surferin an der Eisbachwelle im Englischen Garten in München. Im Moment darf niemand mehr ins Wasser. Aber ist Surfen dort grundsätzlich gefährlich? Gefährlicher als Surfen im Meer?

Zum Artikel: "Schwer Glück gehabt": Surfer erzählt von eigenem Unfall an Isar
Erfahrene Eisbach-Surfer sagen, es sei ein großer Unterschied, ob man in einem Fluss surft oder im Meer. Die Welle komme aus der anderen Richtung und habe einen Auftrieb, sagt Sophia. Sie surft seit neun Jahren an der Eisbachwelle und steht dort zwei bis drei Mal in der Woche auf dem Surfbrett.

Eisbachwelle kein Anfänger-Spot

Die Eisbachwelle sei aber beileibe keine Surf-Spot für Anfänger, erklärt der BR-Journalist Mike Lingenfelser, der dort seit den Anfängen der Welle in den 1980er-Jahren surft. Nur erfahrene Surfer sollten dort ins Wasser gehen, sagt er. Ganz wichtig ist für ihn dabei der Umgang mit der Leash, also der Sicherungsleine. Die Leash ist eine Leine, 

# Detect languages

You can also use the Translation API to detect the language of a text string.

In [13]:
def detect_language(text: str) -> translate.DetectedLanguage:
    client = translate.TranslationServiceClient()

    response = client.detect_language(parent=PARENT, content=text)

    return response.languages[0]


In [14]:
sentences = [
    "Selam Dünya!",
    "Hallo Welt!",
    "¡Hola Mundo!",
    "Ciao mondo!",
    "Γεια σου Κόσμο!",
    "你好世界！",
    "「こんにちは世界」",
    "안녕하세요!",
]
for sentence in sentences:
    language = detect_language(sentence)
    confidence = language.confidence
    language_code = language.language_code
    print(
        f"Confidence: {confidence:4.0%}",
        f"Language: {language_code:5}",
        sentence,
        sep=" | ",
    )
    

Confidence: 100% | Language: tr    | Selam Dünya!
Confidence:  79% | Language: de    | Hallo Welt!
Confidence: 100% | Language: es    | ¡Hola Mundo!
Confidence: 100% | Language: it    | Ciao mondo!
Confidence: 100% | Language: el    | Γεια σου Κόσμο!
Confidence: 100% | Language: zh-CN | 你好世界！
Confidence: 100% | Language: ja    | 「こんにちは世界」
Confidence: 100% | Language: ko    | 안녕하세요!
